In [1]:
import h5py
import numpy as np
from pathlib import Path
from scipy.spatial import cKDTree
import time
from shutil import copyfile
from scorr.addons import group_name

In [2]:
# DIR_PROJECT = Path.home() / "Desktop" / "smoothing_source"
DIR_PROJECT = Path.home() / "Desktop" / "smoothing_source_csem"
file_kernel = DIR_PROJECT / "input_original" / "source_kernel_10003.h5"
file_static_field = DIR_PROJECT / "static_model" / "model_static.h5"

In [3]:
# read kernel file: get coordinates and kernel
with h5py.File(str(file_kernel), "r") as hdf5:
    n_elements_global = hdf5[group_name + "coordinates"].shape[0]
    coordinates_shape = hdf5[group_name + "coordinates"].shape
    coordinates_kernel = np.reshape(hdf5[group_name + "coordinates"][:],
                                    (coordinates_shape[0] * coordinates_shape[1], 3))
    kernel = hdf5[group_name + "distribution"][:]

# read static field: get coordinates and size for distribution
with h5py.File(str(file_static_field), "r") as hdf5:
    assert n_elements_global == hdf5["/ELASTIC/coordinates"].shape[0]
    coordinates_shape = hdf5["/ELASTIC/coordinates"].shape
    coordinates_static_field = np.reshape(hdf5["/ELASTIC/coordinates"][:],
                                          (coordinates_shape[0] * coordinates_shape[1], 3))
    static_field = np.zeros((hdf5["/ELASTIC/data"].shape[1], 125))

In [4]:
# percentile clipping (if wanted)
percentile = 99
indices = abs(kernel) > np.percentile(np.abs(kernel), percentile)
kernel[indices] = kernel[indices]/np.abs(kernel[indices]) * np.percentile(np.abs(kernel), percentile)


In [5]:
# tree = cKDTree(coordinates_static_field)
# 
# start_time = time.time()
# for i in range(coordinates_kernel.shape[0]):
#     distance, ndx = tree.query(coordinates_kernel[i, :], k=4)
#     ndx = ndx[0:np.count_nonzero(distance == 0.0)]
#     static_field[ndx//125, np.mod(ndx, 125)] = kernel[i//25, np.mod(i, 25)]
#         
# end_time = time.time()
# print("total time: ", end_time - start_time)


In [6]:
tree = cKDTree(coordinates_kernel)

start_time = time.time()
for i in range(coordinates_static_field.shape[0]):
    distance, ndx = tree.query(coordinates_static_field[i, :], k=1)
    static_field[i//125, np.mod(i, 125)] = kernel[ndx//25, np.mod(ndx, 25)]
        
end_time = time.time()
print("total time: ", end_time - start_time)

total time:  134.73398995399475


In [7]:
# change static field: kernel acts as initial condition diffusion
file_static_field_new = DIR_PROJECT / "initial_values" / "model_initial_values.h5"
# copyfile(file_static_field, file_static_field_new)
with h5py.File(str(file_static_field), "r") as hdf5_input:
    with h5py.File(str(file_static_field_new), "w") as hdf5_output:
        hdf5_output.create_dataset("/ACOUSTIC/connectivity", shape=hdf5_input["/ELASTIC/connectivity"].shape, 
                            dtype="int32", data=hdf5_input["/ELASTIC/connectivity"][:])
        hdf5_output.create_dataset("/ACOUSTIC/coordinates", shape=hdf5_input["/ELASTIC/coordinates"].shape, 
                            dtype="float32", data=hdf5_input["/ELASTIC/coordinates"][:])
        hdf5_output.create_dataset("/ACOUSTIC/data", shape=static_field.shape, 
                            dtype="float32", data=static_field)
        hdf5_output.create_dataset("/ACOUSTIC/globalElementIds", shape=hdf5_input["/ELASTIC/globalElementIds"].shape, 
                            dtype="int32", data=hdf5_input["/ELASTIC/globalElementIds"][:])
        hdf5_output.create_dataset("/ACOUSTIC/time", shape=hdf5_input["/ELASTIC/time"].shape, 
                            dtype="float32", data=hdf5_input["/ELASTIC/time"][:])


#########################################
############# RUN SMOOTHING #############
#########################################


In [8]:
# allocate array for smooth kernel
kernel_smooth = np.zeros_like(kernel)

# read static field, i.e. smoothed version of kernel
file_static_field_smooth = DIR_PROJECT / "model_smooth.h5"
with h5py.File(str(file_static_field_smooth), "r") as hdf5:
    coordinates_static_field = np.reshape(hdf5["/ACOUSTIC/coordinates"][:],
                                          (coordinates_shape[0] * coordinates_shape[1], 3))
    static_field_smooth = hdf5["/ACOUSTIC/data"][4, :, 0, :]

In [10]:
tree = cKDTree(coordinates_static_field)

start_time = time.time()
for i in range(coordinates_kernel.shape[0]):
    distance, ndx = tree.query(coordinates_kernel[i, :], k=1)
    
    assert distance == 0.0
    kernel_smooth[i//25, np.mod(i, 25)] = static_field_smooth[ndx//125, np.mod(ndx, 125)]
        
end_time = time.time()
print("total time: ", end_time - start_time)


total time:  28.025219917297363


In [11]:
# write smoothed kernel to file
file_kernel_smooth = DIR_PROJECT / "output_smooth" / (file_kernel.stem + "_smooth.h5") 
copyfile(file_kernel, file_kernel_smooth)
with h5py.File(str(file_kernel_smooth), "a") as hdf5:
    hdf5[group_name + "distribution"][:] = kernel_smooth